In [272]:
import re
import os
import asyncio

from tqdm import tqdm

from telethon import TelegramClient

import numpy as np
import pandas as pd

In [2]:
#ENVIRONMENTAL VARIABLES
DATASET_PATH = os.getenv("NER_DATASET_PATH")
BOT_USERNAME = os.getenv("BOT_USERNAME")
API_ID = os.getenv("API_ID")
API_HASH = os.getenv("API_HASH")

### Задание 1
- **1.1.1** Задача с точки зрения NLP состоит в обработке и трансформации текстовых данных в нужный (обычно численный) формат для применения методов ML.

- **1.2.1** Задача извлечения сущностей представляет собой распознавание частей текста (скорее всего обработанного), которые представляют собой сущности (что такое сущности определется постановкой задачи), и их классификацию по средством присваивания им метки из заранее определенного набора сущностных меток (например, человек, организация, глагол и тд)

- **1.2.2** Классические методы: метод, основанный на использовании регулярных выражений (работает непосредственно с текстом). Методы основанные на классическом ML (численная обработка не связанная напрямую с нейронными сетями). К классическому ML можно отнести такие подходы как вероятностные и CRF модели.

- **1.3.1** Задачу NER с помощью LLM можно решать с помощью подхода prompt-engineering'a используя разные тактики (CoT, in-context, few-shot learning и тд.); Если позволяют ресурсы можно слегка (LoRA) дообучить модель на конкретной downstream задаче (в данном случае NER)

- **1.4.1** Зависит от постановки задачи. Обычно оценка задачи проводится по классическим метрикам классификации accuracy, precision, recall, f1-score. Если в задаче используются LLM то часто также считется CER схожесть одной сырой строки с другой.  

### Задание 2

In [269]:
def get_dataset(dataset_path: str = DATASET_PATH) -> pd.DataFrame:
    """
    Loads data into dataframe by folder path
    
    Parameters
    ----------
    dataset_path: str
        Path to dataset folder (folder that occurs after unzipping sample data)
    """
    # Creating list to store resulting frame rows
    row_list = []
    # Getting generators to iterate over paths
    raw_docs_path_generator = os.listdir(DATASET_PATH+'raw/ru')
    annotated_answer_path_generator = os.listdir(DATASET_PATH+'annotated/ru')

    for raw_doc_path in raw_docs_path_generator:
        for annotated_answer_path in annotated_answer_path_generator:
            # Get ids to match docs and gold answers  
            raw_doc_id = re.findall(r"[0-9]+", raw_doc_path)[0]
            annotated_answer_id = re.findall(r"[0-9]+", annotated_answer_path)[0]

            if raw_doc_id == annotated_answer_id:
                # Read document path to get raw text
                with open(DATASET_PATH+'raw/ru/'+raw_doc_path, 'r') as file:
                    raw_data_content = file.read()
                # Read golden answer lines to get entities and ground truths
                with open(DATASET_PATH+'annotated/ru/'+annotated_answer_path, 'r') as file:
                    annotated_answer_lines = file.readlines()
                
                # Normalize answer spaces and capitalization
                # Starting from second line to avoid ID line
                # Transform list of lines to a list of word lists
                splitted_answer_lines = [re.split(r"\s+",line.lower()) for line in annotated_answer_lines[1:]]
                empty_removed_answer_lines = [[word for word in line if word != ''] for line in splitted_answer_lines] 
                # Separate with tabs and spaces
                result_list_lines = []
                for one_word_list_line in empty_removed_answer_lines:
                    N = len(one_word_list_line)
                    # Calculate where the tab should be
                    tab_indx = (N-2)//2 
                    # Prapare for regular space separation
                    space_sep_list = [one_word_list_line[:tab_indx],
                                      one_word_list_line[tab_indx:N-2]]
                    # Prepare for tab separation
                    tab_sep = [' '.join(word_list) for word_list in space_sep_list] + one_word_list_line[-2:]
                    result_list_lines.append('\t'.join(tab_sep))
                
                # Join lines with new lines to return normalized golden answer 
                golden_answer = '\n'.join(result_list_lines)
                # Find entities from golden answer to add to row
                entity = re.findall(r"\s[loc|per|org|evt|pro]+\s", golden_answer)
                stripped_entity = [ent.strip() for ent in entity]

                # Creating and appending to row_list for further df creation
                row_to_append = {
                    "document_id": raw_doc_id,
                    "document_text": raw_data_content,
                    "entity": stripped_entity,
                    "golden_answer": golden_answer
                }
                row_list.append(row_to_append)
    return pd.DataFrame(row_list)

In [270]:
dataset = get_dataset()
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   document_id    9 non-null      object
 1   document_text  9 non-null      object
 2   entity         9 non-null      object
 3   golden_answer  9 non-null      object
dtypes: object(4)
memory usage: 420.0+ bytes
None


In [271]:
dataset.head(3)

,document_id,document_text,entity,golden_answer
0,1006,ru-1006\nru\n2018-07-09\nhttp://polit.ru/news/...,"[evt, pro, per, per, loc, loc, per, per, per, ...",brexit\tbrexit\tevt\tevt-brexit\nfacebook\tfac...
1,1003,ru-1003\nru\n2018-07-09\nhttps://echo.msk.ru/n...,"[per, loc, loc, per, per, org, per, org, per, ...",борис джонсон\tборис джонсон\tper\tper-boris-j...
2,1017,ru-1017\nru\n2018-07-09\nhttp://www.unn.com.ua...,"[evt, pro, per, per, per, per, loc, per, per, ...",brexit\tbrexit\tevt\tevt-brexit\nthe guardian\...
